In [3]:
import xml.etree.ElementTree as ET
from ultralytics import YOLO
from pathlib import Path
import random
import shutil
import cv2
import os

In [ ]:
# Paths
xml_file = '/home/student/Desktop/current_model_ai/annotations.xml'  # Change this to your XML file path
images_folder = '/home/student/Desktop/current_model_ai/ImagesTotal'  # Change this to your images folder path
output_folder = '/home/student/Desktop/current_model_ai'  # Change this to your desired output folder

# Create output directories and clear existing labels
for subset in ['train', 'val', 'test']:
    images_path = os.path.join(output_folder, subset, 'images')
    labels_path = os.path.join(output_folder, subset, 'labels')

    os.makedirs(images_path, exist_ok=True)
    os.makedirs(labels_path, exist_ok=True)

    # Clear any existing labels
    for file in os.listdir(labels_path):
        if file.endswith(".txt"):
            os.remove(os.path.join(labels_path, file))

# Parse the XML file
tree = ET.parse(xml_file)
root = tree.getroot()

images_data = []

for image in root.findall('image'):
    img_name = image.get('name')
    image_width = int(image.get('width'))
    image_height = int(image.get('height'))
    annotations = []

    for box in image.findall('box'):
        label = box.get('label')
        if label == 'V':  # Only include label 'V'
            cls = 0  # 0 for V
            xtl = float(box.get('xtl')) / image_width
            ytl = float(box.get('ytl')) / image_height
            xbr = float(box.get('xbr')) / image_width
            ybr = float(box.get('ybr')) / image_height

            x_center = (xtl + xbr) / 2
            y_center = (ytl + ybr) / 2
            width = xbr - xtl
            height = ybr - ytl

            annotations.append(f'{cls} {x_center} {y_center} {width} {height}')

    # Only add images with annotations (ignores empty annotations)
    if annotations:
        images_data.append((img_name, annotations))

# Shuffle and split
random.shuffle(images_data)
num_images = len(images_data)
train_split = int(0.7 * num_images)
val_split = int(0.9 * num_images)

train_data = images_data[:train_split]
val_data = images_data[train_split:val_split]
test_data = images_data[val_split:]

# Function to save images and annotations
def save_data(data, subset):
    for img_name, annotations in data:
        # Copy image
        src_img = os.path.join(images_folder, img_name)
        dst_img = os.path.join(output_folder, subset, 'images', img_name)
        if os.path.exists(src_img):
            shutil.copy(src_img, dst_img)

        # Save annotation if not empty
        label_file = os.path.join(output_folder, subset, 'labels', img_name.replace('.png', '.txt').replace('.jpg', '.txt'))
        if annotations:
            with open(label_file, 'w') as file:
                for annotation in annotations:
                    file.write(annotation + '\n')

# Save to each subset
save_data(train_data, 'train')
save_data(val_data, 'val')
save_data(test_data, 'test')

print("✅ Data split and saved successfully.")

✅ Data split and saved successfully. Labels with 'P' are completely removed.


In [7]:

# Define the training output folder path
train_folder = "/home/student/Desktop/current_model_ai/folder_train/folds_retrain"

# Remove the folder if it exists
if os.path.exists(train_folder):
    shutil.rmtree(train_folder)

# Load the model
model = YOLO('yolov8l.pt')

# Train the model
model.train(
    data='/home/student/Desktop/current_model_ai/data.yaml',
    epochs=100,
    imgsz=640,
    batch=8,
    project='/home/student/Desktop/current_model_ai/folder_train',
    name='folds_retrain',
    augment=True,  # Enables default augmentations
    degrees=10,
    scale=0.5,
    shear=10,
    perspective=0.0005,
    flipud=0.1,
    fliplr=0.5,
    hsv_h=0.015,
    hsv_s=0.7,
    hsv_v=0.4
)

New https://pypi.org/project/ultralytics/8.3.134 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.133 🚀 Python-3.10.16 torch-2.6.0+cu124 CUDA:0 (NVIDIA RTX 4000 Ada Generation, 20040MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/home/student/Desktop/current_model_ai/data.yaml, degrees=10, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.1, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8l.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=folds_retra

train: Scanning /home/student/Desktop/current_model_ai/train/labels.cache... 1183 images, 485 backgrounds, 0 corrupt: 100%|██████████| 1668/1668 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2771.0±1661.3 MB/s, size: 339.8 KB)


val: Scanning /home/student/Desktop/current_model_ai/val/labels.cache... 338 images, 637 backgrounds, 0 corrupt: 100%|██████████| 975/975 [00:00<?, ?it/s]


...

                   all        975        484      0.296       0.86      0.286      0.117



100 epochs completed in 1.194 hours.
Optimizer stripped from /home/student/Desktop/current_model_ai/folder_train/folds_retrain/weights/last.pt, 87.6MB
Optimizer stripped from /home/student/Desktop/current_model_ai/folder_train/folds_retrain/weights/best.pt, 87.6MB

Validating /home/student/Desktop/current_model_ai/folder_train/folds_retrain/weights/best.pt...
Ultralytics 8.3.133 🚀 Python-3.10.16 torch-2.6.0+cu124 CUDA:0 (NVIDIA RTX 4000 Ada Generation, 20040MiB)
Model summary (fused): 112 layers, 43,607,379 parameters, 0 gradients, 164.8 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 61/61 [00:11<00:00,  5.37it/s]


                   all        975        484      0.309      0.849      0.321       0.14
Speed: 0.1ms preprocess, 10.6ms inference, 0.0ms loss, 0.4ms postprocess per image
Results saved to /home/student/Desktop/current_model_ai/folder_train/folds_retrain


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f141da4cfa0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

In [15]:
# Define the path to the prediction output folder
predict_folder = "/home/student/Desktop/current_model_ai/runs/predict"

# Remove the folder if it exists
if os.path.exists(predict_folder):
    shutil.rmtree(predict_folder)

# Load the model
model = YOLO('/home/student/Desktop/current_model_ai/folder_train/folds_retrain/weights/best.pt')

# Run predictions
results = model.predict(
    source='/home/student/Desktop/current_model_ai/test/images',
    conf=0.3,
    save=True,
    save_txt=True,
    project='/home/student/Desktop/current_model_ai/runs',
    name='predict',
    save_conf=True  # Save confidence values in the txt output
)


image 1/571 /home/student/Desktop/current_model_ai/test/images/Flipped_horizontal_10.png: 480x640 1 V, 10.2ms
image 2/571 /home/student/Desktop/current_model_ai/test/images/Flipped_horizontal_100.png: 480x640 1 V, 8.5ms
image 3/571 /home/student/Desktop/current_model_ai/test/images/Flipped_horizontal_112.png: 480x640 2 Vs, 9.7ms
image 4/571 /home/student/Desktop/current_model_ai/test/images/Flipped_horizontal_123.png: 480x640 2 Vs, 9.2ms
image 5/571 /home/student/Desktop/current_model_ai/test/images/Flipped_horizontal_124.png: 480x640 3 Vs, 8.3ms
image 6/571 /home/student/Desktop/current_model_ai/test/images/Flipped_horizontal_126.png: 480x640 2 Vs, 9.2ms
image 7/571 /home/student/Desktop/current_model_ai/test/images/Flipped_horizontal_128.png: 480x640 1 V, 8.2ms
image 8/571 /home/student/Desktop/current_model_ai/test/images/Flipped_horizontal_129.png: 480x640 2 Vs, 8.5ms
image 9/571 /home/student/Desktop/current_model_ai/test/images/Flipped_horizontal_13.png: 480x640 1 V, 9.5ms
image

In [14]:

# --- CONFIG ---
image_dir = "/home/student/Desktop/current_model_ai/test/images"
pred_label_dir = "/home/student/Desktop/current_model_ai/runs/predict/labels"
output_dir = "/home/student/Desktop/current_model_ai/folder_test_output/visualized_boxes"

os.makedirs(output_dir, exist_ok=True)

image_paths = list(Path(image_dir).glob("*.jpg")) + list(Path(image_dir).glob("*.png"))

for img_path in image_paths:
    image = cv2.imread(str(img_path))
    if image is None:
        continue
    height, width = image.shape[:2]

    # Store boxes and their confidences
    boxes = []

    # === Read Predicted Labels ===
    pred_file = Path(pred_label_dir) / (img_path.stem + ".txt")
    if pred_file.exists():
        with open(pred_file, "r") as f:
            for line in f:
                cls, x, y, w, h, conf = map(float, line.strip().split())
                x_pixel = int(x * width)
                y_pixel = int(y * height)

                # Calculate pixel width and height
                box_width = int(w * width)
                box_height = int(h * height)

                half_w = box_width // 2
                half_h = box_height // 2
                top_left = (x_pixel - half_w, y_pixel - half_h)
                bottom_right = (x_pixel + half_w, y_pixel + half_h)

                # Add box to the list
                boxes.append((cls, top_left, bottom_right, x_pixel, y_pixel, conf))

    # Sort boxes by confidence (highest first)
    boxes.sort(key=lambda x: x[5], reverse=True)

    # Track non-overlapping boxes
    drawn_boxes = []

    for i, (cls, top_left, bottom_right, x_pixel, y_pixel, conf) in enumerate(boxes):
        # Skip overlapping boxes
        overlap = False
        for _, ex_top_left, ex_bottom_right, _, _, _ in drawn_boxes:
            if (top_left[0] < ex_bottom_right[0] and bottom_right[0] > ex_top_left[0] and
                top_left[1] < ex_bottom_right[1] and bottom_right[1] > ex_top_left[1]):
                overlap = True
                break

        if not overlap:
            drawn_boxes.append((cls, top_left, bottom_right, x_pixel, y_pixel, conf))

            center = (x_pixel, y_pixel)

            # Draw the box
            cv2.rectangle(image, top_left, bottom_right, (255, 0, 0), 2)

            # Main dot: blue for top cls 0, red otherwise
            is_blue_dot = i == 0 and cls == 0
            dot_color = (255, 0, 0) if is_blue_dot else (0, 0, 255)
            cv2.circle(image, center, radius=5, color=dot_color, thickness=-1)

            # Show confidence text
            cv2.putText(
                image, f"{conf:.2f}",
                (top_left[0], top_left[1] - 5),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.5, (255, 255, 255), 1
            )

    # Save output image
    out_path = Path(output_dir) / img_path.name
    cv2.imwrite(str(out_path), image)

print("✅ Done! Visualized only center dots and bounding boxes. Pink dots removed.")


✅ Done! Visualized only center dots and bounding boxes. Pink dots removed.
